In [1]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame
import math 

'''
make the matrix
'''
def matrix(ann_matrix):
    annotask = ann_matrix.values.tolist()
    #column_name = list(ann[2].unique())
    matrix = DataFrame(index =ann_matrix[ann_matrix.columns[1]].unique())
    matrix_id = DataFrame(index =ann_matrix[ann_matrix.columns[1]].unique())
    column_name=[]
    for elelist in annotask:
        for i in range(0,len(matrix.index)):
            if elelist[1] == matrix.index[i]:
                if elelist[2] not in column_name:
                    column_name.append(elelist[2])  # list of column_name + that name 
                    matrix[elelist[2]] =0 # dataframe add a column of zero with the name of elelist[2]
                colname_index = column_name.index(elelist[2]) # get the index of the name in the list of column name 
                matrix.iloc[i,colname_index] +=1 #+1 at that cell 
    matrix.index = range(0,len(matrix))
    return matrix, matrix_id  # each value of matrix <=4 

def total(matrix):
    total = 0
    for i in range(0,len(matrix)): 
        total = total+ sum(matrix.iloc[i])
    return total


'''
## compute p_j
# The value p_j is the proportion of all assignments that were made to the jth category(column).
# sum of p_j = 1.0
'''
def compute_p_j(orimatrix):
    p_j=[]
    sum_each_oricol = orimatrix.sum(axis=0)
    for i in sum_each_oricol:
        #p_j.append(i/(len(orimatrix.index)*4))                  #annotator = 4
        p_j.append(i/(len(orimatrix.index)*2))              #   annotator = 2
    suquare_p_j = [i*i for i in p_j]
    aveP_e = sum(suquare_p_j)
    return aveP_e

'''
## Compute P_i 
# the extent to which raters agree for the i-th subject.
# -- how many rater-rater pairs are in agreement,
# relative to the num of all possible rater-rater paires.
'''
def compute_P_i(orimatrix):
    #n = 4                                                      #annotator = 4
    n = 2                                                  #    annotator = 2
    P_i = []
    for i in range(0,len(orimatrix)):
        row_list =[]
        for j in range(0,len(orimatrix.columns)):
            row_list.append(orimatrix.iloc[i][j]*orimatrix.iloc[i][j])
        row_sum = sum(row_list)    
        P_i.append((row_sum-n)/(n*(n-1)))
 
    aveP_i = sum(P_i)/len(P_i)
    return aveP_i

'''
## Compute Kraemer's kappa
'''
def Kraemers_kappa(orimatrix,aveP_e,aveP_i):
    N = len(orimatrix)
    #n = 4                                                     #annotator = 4
    n = 2                                                 #    annotator = 2
    k_0 = ((aveP_i-aveP_e)/(1-aveP_e))+((1-aveP_i)/(N*n*(1-aveP_e)))
    return k_0

'''
## Compute Fleiss' kappa
'''
def Fleiss_kappa(orimatrix,aveP_e,aveP_i):
    #n = 4                                                     #annotator = 4
    n = 2                                                  #   annotator = 2
    k_1 = (aveP_i-aveP_e)/(1-aveP_e)
    return k_1

'''
## Compute Krippendroffis alpha
'''
def Krippendroffis_alpha(orimatrix):
    return None

'''
## split the matrix as dfs 
# where each id is a df, 
# where each error type is a 
# where each column is two, select(y) or non_select(n)
'''
def split_matrix(matrix):
    F_kappa =[]
    t_matrix = matrix.T
    for i in range(0,len(matrix)):
        i = t_matrix.columns[i]
        df = pd.DataFrame(columns = ['selected', 'non_selected'], index =list(matrix.columns))
        df['selected'] = 0
        df['selected'] = t_matrix.iloc[:,i:i+1]
        #df['non_selected'] = 4 - df['selected']
        df['non_selected'] = 2 - df['selected']
        #print(df)
        aveP_e = compute_p_j(df)
        aveP_i = compute_P_i(df)
        F_k = Fleiss_kappa(df,aveP_e,aveP_i)
        F_kappa.append(F_k)
    return F_kappa

---

In [ ]:
import os
for dirname, _, filenames in os.walk('/Users/tianyisun/Desktop/NLQbatches/batch5'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
'''Fleiss_kappa for error_category - sql_id'''
errorcategory = pd.read_csv('/Users/tianyisun/Desktop/NLQbatches/batch5/correct_batch5_anntask_sql_errorcategory.csv')
errorcategory =errorcategory.drop(columns=['Unnamed: 0'], axis=1)
errorcategory = errorcategory[['username','sql_id','code']]
print('length: ', len(errorcategory.sql_id.unique()))
'''calculate Fleiss_kappa'''
errorcategory_matrix,errorcategory_id = matrix(errorcategory)
ls = split_matrix(errorcategory_matrix)
ls = [1 if math.isnan(x) else x for x in ls]
catsign= np.sign(ls)
errorcategory_matrix_ave_F_kappa = sum(ls)/len(ls)
errorcategory_id['category_Fleiss_kappa'] = ls
result = pd.DataFrame([[errorcategory_matrix_ave_F_kappa]], columns=['category_Fleiss_kappa'],index = ['average'])
result = errorcategory_id.append(result)
result

length:  137


,category_Fleiss_kappa
746,1.000000
1088,1.000000
1317,0.607143
548,0.811966
159,1.000000
...,...
332,1.000000
719,1.000000
1402,1.000000
192,-0.100000


In [ ]:
result.to_csv('batch5/correct_batch5_sql-category_Fleiss_kappa.csv')

In [ ]:
### '''Fleiss_kappa for error_type - sql_id'''
errortype = pd.read_csv('/Users/tianyisun/Desktop/NLQbatches/batch7/correct_batch4567_anntask_sql_type.csv')
errortype =errortype.drop(columns=['Unnamed: 0'], axis=1)
errortype = errortype[['username','sql_id','code']]
print('length: ', len(errortype.sql_id.unique()))
'''calculate Fleiss_kappa'''
errortype_matrix,errortype_id = matrix(errortype)
ls = split_matrix(errortype_matrix)
ls = [1 if math.isnan(x) else x for x in ls]
#typesign = np.sign(ls)
#for index in range(0,len(ls)): 
#    if typesign[index]!=catsign[index]:
#        ls[index] = -ls[index]
#    if 0< ls[index]<0.1:
#        ls[index] =ls[index]*10
errortype_matrix_ave_F_kappa = sum(ls)/len(ls)
errortype_id['type_Fleiss_kappa'] = ls
result2 = pd.DataFrame([[errortype_matrix_ave_F_kappa]], columns=['type_Fleiss_kappa'],index = ['average'])
result2 = errortype_id.append(result2)
print(len(result2))
result2

In [ ]:
result2.to_csv('batch7/correct_batch4567_sql-type_Fleiss_kappa.csv')

In [3]:
'''Fleiss_kappa for error_category '''
disagreement = errorcategory[['username','code','sql_id']]

'''calculate Fleiss_kappa'''
disagreement_matrix,id = matrix(disagreement)
ls = split_matrix(disagreement_matrix)
ls = [1 if math.isnan(x) else x for x in ls]
disagreement_matrix_ave_F_kappa = sum(ls)/len(ls)
id['Fleiss_kappa'] = ls
disagreement_result = pd.DataFrame([[disagreement_matrix_ave_F_kappa]], columns=['Fleiss_kappa'],index = ['average'])
#print(len(disagreement_result))
disagreement_result = id.append(disagreement_result)
disagreement_result

/var/folders/gh/3pd23sd51hn2p7774mv2zt4r0000gn/T/ipykernel_4835/3944299363.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  matrix[elelist[2]] =0 # dataframe add a column of zero with the name of elelist[2]


,Fleiss_kappa
B,0.718792
L,0.807584
F,0.768385
A,0.790883
N,0.725852
C,0.708657
E,0.882107
D,0.780975
K,0.857613
P,0.796283


In [4]:
disagreement_result.to_csv('batch5/correct_batch5_category_Fleiss_kappa.csv')

---
---

'''
## preparing for matrix 
'''
mapping= pd.read_csv('/kaggle/input/forkappa/mapping.csv')
#record_id = pd.read_csv('/kaggle/input/matrix/record_id_code.csv')
#description = pd.read_csv('/kaggle/input/matrix2/description_matrx.csv')
#descri_matrix= description.iloc[0:181,1:39]
#record_matrix= record_id.iloc[0:184,1:39]

---
---